In [1]:
# Jupyter magic
%matplotlib inline

# Standard library imports
import os
import time
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

# Third-party imports
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import BallTree
from numba import njit, prange


ModuleNotFoundError: No module named 'polars'

In [2]:
def travail_lourd(x):
    if x == 5:
        raise ValueError("Oups, erreur volontaire pour x=5")
    time.sleep(1)
    return x * x

inputs = list(range(12))
results = []

max_workers = min(4, os.cpu_count() or 1)

with ThreadPoolExecutor(max_workers=max_workers) as exe:
    futures = {exe.submit(travail_lourd, i): i for i in inputs}
    for fut in as_completed(futures):
        i = futures[fut]
        try:
            res = fut.result()
        except Exception as e:
            print(f"Tâche {i} a levé une exception : {e}")
            res = None
        results.append(res)

print("Résultats :", results)


Tâche 5 a levé une exception : Oups, erreur volontaire pour x=5
Résultats : [0, 4, None, 1, 9, 16, 36, 49, 64, 81, 121, 100]


In [3]:
@njit(parallel=True)
def somme_racines(n):
    tmp = np.zeros(n)
    for i in prange(n):
        tmp[i] = np.sqrt(i)
    return np.sum(tmp)


### Import DATASET Data_sales

In [4]:
####################### DATA MACRO ECO - DVF  / DEMAND #############################
## paths
#folder_path_M = '/Users/maximehenon/Documents/GitHub/MAR25_BDS_Compagnon_Immo/'
#folder_path_Y = 'C:/Users/charl/OneDrive/Documents/Yasmine/DATASCIENTEST/FEV25-BDS-COMPAGNON'
#folder_path_C = '../data/processed/Sales'
# folder_path_L= '/Users/loick.d/Documents/Datascientest/Github immo/MAR25_BDS_Compagnon_Immo/'
folder_path_LW = 'C:/Users/User/Downloads/drive-download-20250508T155351Z-1-001'


# Load the dataset df_sales_clean / OFFER
#output_file1 = os.path.join(folder_path_M, 'merged_sales_data.csv')
#output_file1 = os.path.join(folder_path_Y, 'merged_sales_data.csv')
#output_file1 = os.path.join(folder_path_C, 'merged_sales_data.csv')
#output_file1 = os.path.join(folder_path_L, 'merged_sales_data.csv')
output_file1 = os.path.join(folder_path_LW, 'merged_sales_data.csv')



####################### DATA SALES / ANNONCES OFFER #############################
chunksize = 100000  # Number of rows per chunk
chunks = pd.read_csv(output_file1, sep=';', chunksize=chunksize, index_col=None, on_bad_lines='skip', low_memory=False)
# Process chunks
df_sales_clean = pd.concat(chunk for chunk in chunks)

## Rappel des colonnes restantes
# print("Colonnes restantes dans le DataFrame :")
# print(df_sales_clean.columns)
# print(df_sales_clean.dtypes)
# print("\nShape du Dataset après élimination des colonnes :", df_sales_clean.shape)
print(df_sales_clean['INSEE_COM'].info())



<class 'pandas.core.series.Series'>
RangeIndex: 9849326 entries, 0 to 9849325
Series name: INSEE_COM
Non-Null Count    Dtype 
--------------    ----- 
9849326 non-null  object
dtypes: object(1)
memory usage: 75.1+ MB
None


### Import DATASET DVF

### Intégration des données effectives socio-économique et des prix de ventes effectifs

Le taux de chomage présenté est celui de 2023. Il y a très peu de variation du taux de chomage en France en 2024 vs 2023 (+0.1pts)

Le décalage des variables macroéconomiques va permettre d'éviter:
-  l’« effet d’anticipation » et capitaliser sur la disponibilité des données. Ainsi, employer le taux de chômage de 2023 comme variable explicative pour des prix observés en 2024 est non seulement acceptable, mais fait partie des bonnes pratiques :

- Disponibilité et robustesse : les statistiques de chômage sortent avec un délai (retard de publication) et présentent souvent des fluctuations intratrimestrielles.

- Pas de fuite de données (“look-ahead bias”) : en régressant les prix 2024 sur le chômage 2023, on garantit que le modèle ne s’appuie pas sur des informations non disponibles au moment où le prix s’est formé.

- Effet retardé des variables macro : de nombreux travaux montrent que le chômage influence les prix immobiliers avec un certain décalage temporel ; on observe souvent un impact court à moyen terme (6–12 mois) puis un ajustement plus long (12–24 mois) selon les marchés (source: PMC)

In [5]:

####################### DATA DVF / DEMAND #############################

# Load the dataset df_sales_clean / DEMAND

#output_file2 = os.path.join(folder_path_M, 'DVF_donnees_macroeco.csv')
#output_file2 = os.path.join(folder_path_Y, 'DVF_donnees_macroeco.csv')
#output_file2 = os.path.join(folder_path_C, 'DVF_donnees_macroeco.csv')
#output_file2 = os.path.join(folder_path_L, 'DVF_donnees_macroeco.csv')
output_file2 = os.path.join(folder_path_LW, 'DVF_donnees_macroeco.csv')


chunksize = 100000  # Number of rows per chunk
chunks = pd.read_csv(output_file2, sep=',', chunksize=chunksize, index_col=None, on_bad_lines='skip', low_memory=False)
# Process chunks
df_dvf = pd.concat(chunk for chunk in chunks)

print(df_sales_clean['INSEE_COM'].dtype)
print(df_dvf['Code INSEE de la commune'].dtype)

# Clean format
df_sales_clean['INSEE_COM'] = df_sales_clean['INSEE_COM'].astype(str).str.pad(width=5, side='left', fillchar='0')
df_dvf['INSEE_COM'] = df_dvf['Code INSEE de la commune'].astype(str)

# Verification
print(df_sales_clean['INSEE_COM'].unique()[:30])
print(df_dvf['INSEE_COM'].unique()[:30])

df_sales_clean['INSEE_COM'] = df_sales_clean['INSEE_COM'].str.strip()
df_dvf['INSEE_COM'] = df_dvf['INSEE_COM'].str.strip()

# Load the dataset df_sales_clean / DEMAND


object
object
['01451' '01014' '01360' '01288' '01160' '01350' '01241' '01175' '01071'
 '01304' '01032' '01247' '01457' '01250' '01187' '01435' '01090' '01239'
 '01426' '01123' '01232' '01096' '01085' '01445' '01180' '01375' '01049'
 '01138' '01108' '01424']
['nan' '10006' '10011' '10018' '10020' '10025' '10030' '10031' '10033'
 '10034' '10035' '10045' '10049' '10051' '10060' '10061' '10064' '10067'
 '10080' '10081' '10083' '10084' '10086' '10094' '10096' '10097' '10098'
 '10100' '10110' '10113']


### nettoyage des données dvf - le score scolaire

In [6]:

#nettoyage des données IPS : on ne garde que la rentrée 2023-2024, le secteur public, et une moyenne IPS par code commune sur primaire et elementaire
# Pour le moment nous ne conserverons que les données primaires
print(df_dvf.columns.tolist())

df_dvf = df_dvf[df_dvf['Rentrée scolaire'] == '2023-2024']
df_dvf = df_dvf[df_dvf['Secteur'] == 'public']

df_schools = df_dvf.drop(columns = ['avg_purchase_price_m2', 'avg_rent_price_m2', 'rental_yield_pct', 'num_ligne'])
df_schools['IPS'] = df_schools['IPS'].astype(str).replace('NS', np.nan)
df_schools['IPS'] = (df_schools['IPS'].astype(str).str.replace(',', '.').astype(float))

# creation d'une colonne "levele = "Primaire" |"elementaire"
def extract_level(name):
    name = name.upper()
    if 'PRIMAIRE' in name:
        return 'primaire'
    if 'ELEMENTAIRE' in name:
        return 'elementaire'
    return np.nan

df_schools['level'] = df_schools['Etablissement'].apply(extract_level)

# Filtrer les lignes valides et grouper pour faire la moyenne par commune et par niveau
df_levels = (df_schools.dropna(subset=['level']).groupby(['INSEE_COM', 'level'])['IPS'].mean().reset_index())

# Pivot pour obtenir une ligne par code_com, deux colonnes distinctes
df_ips_par_commune = (df_levels.pivot(index='INSEE_COM', columns='level', values='IPS').rename(columns={
        'primaire': 'IPS_primaire',
        'elementaire': 'IPS_elementaire'
    }).reset_index())

# Affichage
print(df_ips_par_commune.head())

# merge
df_ips_small = df_ips_par_commune[['INSEE_COM', 'IPS_primaire']].drop_duplicates('INSEE_COM')
df_dvf_small = df_dvf[['avg_purchase_price_m2', 'avg_rent_price_m2', 'rental_yield_pct', 'INSEE_COM']].drop_duplicates('INSEE_COM')




# Dict de lookup pour chaque colonne
purchase_map = dict(zip(df_dvf_small['INSEE_COM'], df_dvf_small['avg_purchase_price_m2']))
rent_map     = dict(zip(df_dvf_small['INSEE_COM'], df_dvf_small['avg_rent_price_m2']))
yield_map    = dict(zip(df_dvf_small['INSEE_COM'], df_dvf_small['rental_yield_pct']))
ips_map = dict(zip(df_ips_small['INSEE_COM'], df_ips_small['IPS_primaire']))


# Mapping dans df_sales_clean (pas de copy de tout le DF)
df_sales_clean['avg_purchase_price_m2'] = df_sales_clean['INSEE_COM'].map(purchase_map)
df_sales_clean['avg_rent_price_m2']     = df_sales_clean['INSEE_COM'].map(rent_map)
df_sales_clean['rental_yield_pct']      = df_sales_clean['INSEE_COM'].map(yield_map)
df_sales_clean['IPS_primaire'] = df_sales_clean['INSEE_COM'].map(ips_map)


# display(df_sales_clean.isna().sum())
# display(df_sales_clean[df_sales_clean['avg_rent_price_m2'].isna()][['INSEE_COM', 'avg_purchase_price_m2', 'dpeL', 'prix_m2_vente']].head(20))
# #display(df_sales_clean[df_sales_clean['INSEE_COM']==]['INSEE_COM', 'avg_purchase_price_m2', 'dpeL', 'prix_m2_vente'].head(20))



['Unnamed: 0', 'code_commune', 'avg_purchase_price_m2', 'avg_rent_price_m2', 'rental_yield_pct', 'num_ligne', 'Rentrée scolaire', 'Code INSEE de la commune', 'Nom de la commune', 'Etablissement', 'Secteur', 'IPS', 'INSEE_COM']
level INSEE_COM  IPS_elementaire  IPS_primaire
0         10006             92.1           NaN
1         10011             97.1           NaN
2         10018              NaN          93.9
3         10020              NaN         112.3
4         10025              NaN          99.0


In [7]:
display(df_sales_clean.columns)

Index(['idannonce', 'type_annonceur', 'typedebien', 'typedetransaction',
       'etage', 'surface', 'surface_terrain', 'nb_pieces', 'prix_bien',
       'prix_maison', 'prix_terrain', 'mensualiteFinance', 'balcon', 'eau',
       'bain', 'dpeL', 'dpeC', 'mapCoordonneesLatitude',
       'mapCoordonneesLongitude', 'annonce_exclusive', 'nb_etages', 'parking',
       'places_parking', 'cave', 'exposition', 'ges_class',
       'annee_construction', 'nb_toilettes', 'nb_terraces', 'videophone',
       'porte_digicode', 'surface_balcon', 'ascenseur', 'nb_logements_copro',
       'charges_copro', 'chauffage_energie', 'chauffage_systeme',
       'chauffage_mode', 'categorie_annonceur', 'logement_neuf', 'duree_int',
       'typedebien_lite', 'date', 'INSEE_COM', 'IRIS', 'CODE_IRIS',
       'TYP_IRIS_x', 'TYP_IRIS_y', 'GRD_QUART', 'UU2010', 'REG', 'DEP',
       'loyer_m2_median_n6', 'nb_log_n6', 'taux_rendement_n6',
       'loyer_m2_median_n7', 'nb_log_n7', 'taux_rendement_n7', 'prix_m2_vente',
    

### Imputation des nan par les données geo les plus proches

In [8]:
# 1. Moyenne lat/lon et IPS_primaire par commune
communes = (
    df_sales_clean
    .groupby('INSEE_COM')
    .agg({
      'mapCoordonneesLatitude':'mean',
      'mapCoordonneesLongitude':'mean',
      'IPS_primaire':'mean'  # NaN restent NaN
    })
    .reset_index()
)

# 2. Séparer communes complètes vs manquantes
complete = communes[communes['IPS_primaire'].notna()].copy()
missing  = communes[communes['IPS_primaire'].isna()].copy()

# 3. NearestNeighbors sur complete (en radians)

coords = np.radians(complete[['mapCoordonneesLatitude','mapCoordonneesLongitude']])
tree = BallTree(coords, metric='haversine')

# 4. Pour chaque commune manquante, chercher la plus proche
coords_m = np.radians(missing[['mapCoordonneesLatitude','mapCoordonneesLongitude']])
dist, idx = tree.query(coords_m, k=1)
missing['IPS_primaire'] = complete.iloc[idx.flatten()]['IPS_primaire'].values

# Concat et merge back sur chaque annonce
communes_filled = pd.concat([complete, missing], ignore_index=True)[
    ['INSEE_COM', 'IPS_primaire']
].drop_duplicates('INSEE_COM')

# dict pour map  
ips_map = dict(zip(
    communes_filled['INSEE_COM'],
    communes_filled['IPS_primaire']
))

df_sales_clean['IPS_primaire_imputed'] = df_sales_clean['INSEE_COM'].map(ips_map)


df_sales_clean['IPS_primaire'] = df_sales_clean['IPS_primaire'].fillna(
    df_sales_clean['IPS_primaire_imputed']
)
df_sales_clean.drop(columns=['IPS_primaire_imputed'], inplace=True)




## Export des données

In [11]:
display(df_sales_clean.head())

# del df_sales_clean['IPS_elementaire']
# del df_sales_clean['num_ligne']

display(df_sales_clean.head())

## Paths
#folder_path_M = '/Users/maximehenon/Documents/GitHub/MAR25_BDS_Compagnon_Immo/'
#folder_path_Y = 'C:/Users/charl/OneDrive/Documents/Yasmine/DATASCIENTEST/FEV25-BDS-COMPAGNON'
#folder_path_C = '../data/processed/Sales'
#folder_path_L = '/Users/loick.d/Documents/Datascientest/Github immo/MAR25_BDS_Compagnon_Immo/'
folder_path_LW = 'C:/Users/User/Downloads/drive-download-20250508T155351Z-1-001'

#### YASMINE ####
# Nettoyage des types de données avant conversion
object_cols = df_sales_clean.select_dtypes(include='object').columns
for col in object_cols:
    df_sales_clean[col] = df_sales_clean[col].astype(str)


# Conversion en DataFrame Polars
df_pl = pl.from_pandas(df_sales_clean)

# Export en Parquet
parquet_path = Path(folder_path_LW) / "df_sales_clean_polars.parquet"
df_pl.write_parquet(parquet_path)
print(f"✅ Export Parquet Polars : {parquet_path}")

# Export en CSV
csv_path = Path(folder_path_LW) / "df_sales_clean_polars.csv"
df_pl.write_csv(csv_path, separator=";")
print(f"✅ Export CSV Polars : {csv_path}")

print("Exports Parquet et CSV Polars terminés.")

,idannonce,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,...,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente,avg_purchase_price_m2,avg_rent_price_m2,rental_yield_pct,IPS_primaire
0,entities-1037638-4560933,pr,m,v,0,79,470.0,4,241000,NaN,...,3.0,0.039,10.42,9.0,0.041,3050.63,NaN,NaN,NaN,110.0
1,immo-facile-57743459,pr,a,v,0,65,650.0,2,136000,NaN,...,3.0,0.061,10.50,11.0,0.060,2092.31,NaN,NaN,NaN,110.0
2,entities-1037666-4560741,pr,m,v,0,114,450.0,5,260900,NaN,...,2.0,0.059,11.34,2.0,0.059,2288.60,NaN,NaN,NaN,111.7
3,immo-facile-57762298,pr,m,v,0,148,1500.0,7,286000,NaN,...,NaN,NaN,7.27,2.0,0.045,1932.43,NaN,NaN,NaN,110.0
4,immo-facile-57762759,pr,m,v,0,213,2251.0,8,435000,NaN,...,NaN,NaN,NaN,NaN,NaN,2042.25,NaN,NaN,NaN,110.0


,idannonce,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,...,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente,avg_purchase_price_m2,avg_rent_price_m2,rental_yield_pct,IPS_primaire
0,entities-1037638-4560933,pr,m,v,0,79,470.0,4,241000,NaN,...,3.0,0.039,10.42,9.0,0.041,3050.63,NaN,NaN,NaN,110.0
1,immo-facile-57743459,pr,a,v,0,65,650.0,2,136000,NaN,...,3.0,0.061,10.50,11.0,0.060,2092.31,NaN,NaN,NaN,110.0
2,entities-1037666-4560741,pr,m,v,0,114,450.0,5,260900,NaN,...,2.0,0.059,11.34,2.0,0.059,2288.60,NaN,NaN,NaN,111.7
3,immo-facile-57762298,pr,m,v,0,148,1500.0,7,286000,NaN,...,NaN,NaN,7.27,2.0,0.045,1932.43,NaN,NaN,NaN,110.0
4,immo-facile-57762759,pr,m,v,0,213,2251.0,8,435000,NaN,...,NaN,NaN,NaN,NaN,NaN,2042.25,NaN,NaN,NaN,110.0


✅ Export Parquet Polars : C:\Users\User\Downloads\drive-download-20250508T155351Z-1-001\df_sales_clean_polars.parquet
✅ Export CSV Polars : C:\Users\User\Downloads\drive-download-20250508T155351Z-1-001\df_sales_clean_polars.csv
Exports Parquet et CSV Polars terminés.


In [10]:
file_path = "C:/Users/User/Downloads/drive-download-20250508T155351Z-1-001/df_sales_clean_polars.csv"

df_test = pd.read_csv(file_path, sep=';', nrows=1000)
print(df_test.head())
print(df_test.columns)
print(df_test.shape)

df_test['date'] = pd.to_datetime(df_test['date'], errors='coerce')
print(df_test['date'].min(), df_test['date'].max())
print(df_test['date'].isna().sum(), "dates non reconnues")



                  idannonce type_annonceur typedebien typedetransaction  \
0  entities-1037638-4560933             pr          m                 v   
1      immo-facile-57743459             pr          a                 v   
2  entities-1037666-4560741             pr          m                 v   
3      immo-facile-57762298             pr          m                 v   
4      immo-facile-57762759             pr          m                 v   

   etage  surface  surface_terrain  nb_pieces  prix_bien  prix_maison  ...  \
0      0       79            470.0          4     241000          NaN  ...   
1      0       65            650.0          2     136000          NaN  ...   
2      0      114            450.0          5     260900          NaN  ...   
3      0      148           1500.0          7     286000          NaN  ...   
4      0      213           2251.0          8     435000          NaN  ...   

   nb_log_n6  taux_rendement_n6  loyer_m2_median_n7  nb_log_n7  \
0        3.0  